# Assignment 1

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: POS tagging, Sequence labelling, RNNs


# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Federico Ruggeri -> federico.ruggeri6@unibo.it
* Eleonora Mancini -> e.mancini@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# Introduction

You are tasked to address the task of POS tagging.

<center>
    <img src="./images/pos_tagging.png" alt="POS tagging" />
</center>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!cp -rf /content/drive/MyDrive/UNIBO/NLP/Assignments/Assignment-1/data ./
#!cp -rf /content/drive/MyDrive/UNIBO/NLP/Assignments/Assignment-1/images ./
#!cp /content/drive/MyDrive/UNIBO/NLP/Assignments/Assignment-1/data.csv ./

In [1]:
seed = 0

# [Task 1 - 0.5 points] Corpus

You are going to work with the [Penn TreeBank corpus](https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip).

**Ignore** the numeric value in the third column, use **only** the words/symbols and their POS label.

### Example

```Pierre	NNP	2
Vinken	NNP	8
,	,	2
61	CD	5
years	NNS	6
old	JJ	2
,	,	2
will	MD	0
join	VB	8
the	DT	11
board	NN	9
as	IN	9
a	DT	15
nonexecutive	JJ	15
director	NN	12
Nov.	NNP	9
29	CD	16
.	.	8
```

### Splits

The corpus contains 200 documents.

   * **Train**: Documents 1-100
   * **Validation**: Documents 101-150
   * **Test**: Documents 151-199

### Instructions

* **Download** the corpus.
* **Encode** the corpus into a pandas.DataFrame object.
* **Split** it in training, validation, and test sets.

In [2]:
import os
import pandas as pd
import numpy as np
import random

random.seed(seed)
np.random.seed(seed)


data_folder = "./data"
def encode_dataset(dataset_name: str, to_lower: bool) -> pd.DataFrame:
  """
    Takes the dataset and encodes it in a pandas dataframe having six columns ['split', 'doc_id', 'sentence_num', 'words', 'tags', 'num_tokens']. Computes also unique tags set and unique words set and returns them with the dataframe.
  
  """
  print("Encoding dataset as pandas dataframe...")

  dataset_folder = os.path.join(data_folder+ "/dataset")
  
  dataframe_rows = []             #dataframe that will contain all the sentences in all the documents, each sentence as a list of word and a list of corresponding tags
  unique_tags = set()             
  unique_words = set()

  for doc in os.listdir(dataset_folder):
    if doc.endswith(".csv") or doc.endswith(".pkl"): continue
    doc_num = int(doc[5:8])
    doc_path = os.path.join(dataset_folder,doc)

    with open(doc_path, mode='r', encoding='utf-8') as file:
      df = pd.read_csv(file,sep='\t',header=None,skip_blank_lines=False)
      df.rename(columns={0:'word',1:"TAG",2:"remove"},inplace=True)
      df.drop("remove",axis=1,inplace=True)

      if to_lower: df['word'] = df["word"].str.lower() #set all words to lower case
      
      #create another column that indicate the group id by sentence 
      df["group_num"] = df.isnull().all(axis=1).cumsum()
      df.dropna(inplace=True)
      df.reset_index(drop=True, inplace=True)
      
      unique_tags.update(df['TAG'].unique())     #save all the unique tags in a set 
      unique_words.update(df['word'].unique())   #save all the unique words in a set 

      #generate sentence list in a document 
      df_list = [df.iloc[rows] for _, rows in df.groupby('group_num').groups.items()]
      for n,d in enumerate(df_list) :           #for each sentence create a row in the final dataframe
          dataframe_row = {
              "split" : 'train' if doc_num<=100 else ('val' if doc_num<=150  else 'test'),
              "doc_id" : doc_num,
              "sentence_num" : n,
              "words": d['word'].tolist(),
              "tags":  d['TAG'].tolist(),
              "num_tokens": len(d['word'])
          }
          dataframe_rows.append(dataframe_row)

  dataframe_path = os.path.join(data_folder, dataset_name)
  df_final = pd.DataFrame(dataframe_rows)
  df_final.to_csv(dataframe_path + ".csv")                      #save as csv to inspect

  print("Encoding completed!")
    
  return  df_final, unique_tags, unique_words

df, unique_tags, unique_words = encode_dataset("encoded_dataset", to_lower = True)

print('Some words from the dataset:', random.choices(list(unique_words),k=15))
print('Some tags from the dataset:', random.choices(list(unique_tags),k=15))

print('\nencoded dataframe:')

Encoding dataset as pandas dataframe...
Encoding completed!
Some words from the dataset: ['producer', 'slowly', 'seduce', 'u.s.a.', 'increasing', 'random', 'forecast', 'dating', 'fanciful', 'overruns', 'lender', 'prepared', 'nets', 'entering', 'profitability']
Some tags from the dataset: ['VBD', 'RB', 'CD', 'VB', 'RB', 'WRB', 'JJS', 'RB', '$', '-LRB-', 'WP$', 'VBN', 'VBP', 'PRP$', 'PDT']

encoded dataframe:


In [3]:
df.sort_values("doc_id").groupby('split').head()

,split,doc_id,sentence_num,words,tags,num_tokens
2370,train,1,0,"[pierre, vinken, ,, 61, years, old, ,, will, j...","[NNP, NNP, ,, CD, NNS, JJ, ,, MD, VB, DT, NN, ...",18
2371,train,1,1,"[mr., vinken, is, chairman, of, elsevier, n.v....","[NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, NNP, ...",13
921,train,2,0,"[rudolph, agnew, ,, 55, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, IN, NNP...",26
453,train,3,0,"[a, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS...",36
454,train,3,1,"[the, asbestos, fiber, ,, crocidolite, ,, is, ...","[DT, NN, NN, ,, NN, ,, VBZ, RB, JJ, IN, PRP, V...",32
623,val,101,1,"[for, the, agency, for, international, develop...","[IN, DT, NNP, IN, NNP, NNP, ,, NNS, VBD, $, CD...",51
636,val,101,14,"[rep., jerry, lewis, ,, a, conservative, calif...","[NNP, NNP, NNP, ,, DT, JJ, NN, ,, VBD, DT, NN,...",53
624,val,101,2,"[the, conference, approved, at, least, $, 55, ...","[DT, NN, VBD, IN, JJS, $, CD, CD, IN, JJ, NN, ...",48
622,val,101,0,"[a, house-senate, conference, approved, major,...","[DT, NNP, NN, VBD, JJ, NNS, IN, DT, NN, IN, JJ...",44
625,val,101,3,"[the, agreement, on, poland, contrasts, with, ...","[DT, NN, IN, NNP, VBZ, IN, DT, JJ, NNS, VBG, I...",37


In [4]:
from collections import OrderedDict
import pickle

dict_path = os.path.join(data_folder,'dictionaries.pkl') #path where dictionaries will be saved 

def build_dict(words : list[str], tags : list[str]): 
    """
        Builds 4 dictionaries word2int, int2word, tag2int, int2tag and returns them
    """
    
    word2int = OrderedDict()
    int2word = OrderedDict()

    for i, word in enumerate(words):
        word2int[word] = i+1           #plus 1 since the 0 will be used as tag token 
        int2word[i+1] = word

    tag2int = OrderedDict()
    int2tag = OrderedDict()

    for i, tag in enumerate(tags):
        tag2int[tag] = i+1
        int2tag[i+1] = tag
    
    print('saving dictionaries as pickle files')
    pickle_files = [word2int,int2word,tag2int,int2tag]
    
    with open(dict_path, 'wb') as f:
        pickle.dump(pickle_files, f)

    return word2int,int2word,tag2int,int2tag

word2int,int2word,tag2int,int2tag = build_dict(unique_words,unique_tags)

saving dictionaries as pickle files


In [5]:
indexed_df_path = os.path.join(data_folder, "indexed_dataset.pkl") #numberized dataframe path

def build_indexed_dataframe(word2int, tag2int, df):
    """
        Given the dictionaries word2int, tag2int and the dataframe, creates a dataframe were every word and tag is represented by its number and returns it
    """
    print('Initiating numberization of words and tags in dataframe')
    indexed_rows = []
    for words,tags in zip(df['words'],df['tags']):
        indexed_row = {'indexed_words':[word2int[word] for word in words ],'indexed_tags':[tag2int[tag] for tag in tags ]}
        indexed_rows.append(indexed_row)
    
    indexed_df = pd.DataFrame(indexed_rows)

    indexed_df.insert(0,'split',df['split'])
    indexed_df.insert(1,'num_tokens',df['num_tokens'])

    print('Numberization completed')

    return indexed_df


def check_dataframe_numberization(indexed_df, normal_df, int2word, int2tag) :
    """
       Checks if the numberized dataframe will lead to the normal dataframe usind the dictionaries int2word and int2tag
    """
    for n, (w_t, t_t) in enumerate(zip(indexed_df['indexed_words'],indexed_df['indexed_tags'])):
        if not normal_df.loc[n,'words'] == [int2word[indexed_word] for indexed_word in w_t]:
            print('words numberization gone wrong') 
            return False
        if not normal_df.loc[n,'tags'] == [int2tag[indexed_tag] for indexed_tag in t_t]:
            print('tags numberization gone wrong')
            return False 
    
    print('\nAll right with dataset numberization')
    print('Saving indexed dataframe')
    
    indexed_df.to_pickle(indexed_df_path)


indexed_df = build_indexed_dataframe(word2int,tag2int,df)
check_dataframe_numberization(indexed_df,df, int2word, int2tag)


Initiating numberization of words and tags in dataframe
Numberization completed

All right with dataset numberization
Saving indexed dataframe


# [Task 2 - 0.5 points] Text encoding

To train a neural POS tagger, you first need to encode text into numerical format.

### Instructions

* Embed words using **GloVe embeddings**.
* You are **free** to pick any embedding dimension.
* [Optional] You are free to experiment with text pre-processing: **make sure you do not delete any token!**

In [6]:
import torch
from torchtext.vocab import GloVe
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

embedding_dimension = 300

glove_embeddings = GloVe(name='6B', dim=embedding_dimension)

In [7]:
def build_embedding_matrix(emb_model, word2int):
    """
        Given the embedding model and the dict. word2int. If there is the embedding for the word, we add it to the embedding_matrix. In negative case we put a list of random values.
        Return the embedding matrix
    """
    #check_value_distribution_glove(emb_model)
   
    embedding_dimension = len(emb_model[0]) #how many numbers each emb vector is composed of                                                           
    embedding_matrix = np.zeros((len(word2int)+1, embedding_dimension), dtype=np.float32)   #create a matrix initialized with all zeros 

    for word, idx in word2int.items():
        if word in emb_model.stoi:
            embedding_matrix[idx] = emb_model[word]
        else:
            embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)
            
    print('Saving embedding matrix')
    path = os.path.join(data_folder, "emb_matrix")
    np.save(path,embedding_matrix,allow_pickle=True)

    print("Embedding matrix shape: {}".format(embedding_matrix.shape))

    return embedding_matrix

embedding_matrix = build_embedding_matrix(glove_embeddings, word2int)

Saving embedding matrix
Embedding matrix shape: (10948, 300)


In [8]:
def load_data():
    """
        Loads the data "emb_matrix, indexed_dataset, word2int, int2word, tag2int, int2tag " and returns them
    """
    emb_matrix_path = os.path.join(data_folder,'emb_matrix.npy')
    indexed_dataset_path = os.path.join(data_folder,'indexed_dataset.pkl')
    dictionaries_path = os.path.join(data_folder,'dictionaries.pkl')

    if os.path.exists(emb_matrix_path) and os.path.exists(indexed_dataset_path):
        print('Loading embedding matrix')
        emb_matrix = np.load(emb_matrix_path,allow_pickle=True)
        print('Loading numberized dataset')
        indexed_dataset = pd.read_pickle(indexed_dataset_path)
        print('Loading dictionaries')
        with open(dictionaries_path, 'rb') as f:
            word2int,int2word,tag2int,int2tag = pickle.load(f)
        
        print('All data loaded')
    else:
        print('What you are looking for is not present in the folder')
        emb_matrix, indexed_dataset = None, None

    return emb_matrix, indexed_dataset, word2int, int2word, tag2int, int2tag

emb_matrix, indexed_dataset, word2int, int2word, tag2int, int2tag = load_data()

Loading embedding matrix
Loading numberized dataset
Loading dictionaries
All data loaded


# [Task 3 - 1.0 points] Model definition

You are now tasked to define your neural POS tagger.

### Instructions

* **Baseline**: implement a Bidirectional LSTM with a Dense layer on top.
* You are **free** to experiment with hyper-parameters to define the baseline model.

* **Model 1**: add an additional LSTM layer to the Baseline model.
* **Model 2**: add an additional Dense layer to the Baseline model.

* **Do not mix Model 1 and Model 2**. Each model has its own instructions.

**Note**: if a document contains many tokens, you are **free** to split them into chunks or sentences to define your mini-batches.

### Embedding layer

In [9]:
import torch.nn as nn
import torch.nn.functional as F

def create_emb_layer(weights_matrix, pad_idx = 0):
    """
        Creates and returns the embedding layer
    """
    matrix = torch.Tensor(weights_matrix)   #the embedding matrix 
    _ , embedding_dim = matrix.shape 
    emb_layer = nn.Embedding.from_pretrained(matrix, freeze=True, padding_idx = pad_idx)   #load pretrained weights in the layer and make it non-trainable 
    return emb_layer, embedding_dim

### Baseline model

In [10]:
class Baseline(nn.Module):
    def __init__(self, lstm_dimension, dense_dimension):
        super().__init__()
        self.bidirectional_layer = nn.LSTM(bidirectional=True, input_size=embedding_dimension, hidden_size=lstm_dimension, batch_first=True)
        self.dense_layer = nn.Linear(in_features=lstm_dimension*2, out_features=dense_dimension)
        self.embedding_layer, self.embedding_dim = create_emb_layer(embedding_matrix)

    def forward(self, sentences, sentences_length):
        embedded_sentences = self.embedding_layer(sentences)
        packed_sentences = nn.utils.rnn.pack_padded_sequence(embedded_sentences, sentences_length, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.bidirectional_layer(packed_sentences)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        output = self.dense_layer(output)
        output = F.log_softmax(output, dim=2)
        return output

        

### Model 1

In [11]:
class Model1(nn.Module):
    def __init__(self, lstm_dimension, dense_dimension):
        super().__init__()
        self.bidirectional_layer_1 = nn.LSTM(bidirectional=True, input_size=embedding_dimension, hidden_size=lstm_dimension, batch_first=True)
        self.bidirectional_layer_2 = nn.LSTM(bidirectional=True, input_size=lstm_dimension*2, hidden_size=lstm_dimension, batch_first=True)
        self.dense_layer = nn.Linear(in_features=lstm_dimension*2, out_features=dense_dimension)
        self.embedding_layer, self.embedding_dim = create_emb_layer(embedding_matrix)

    def forward(self, sentences, sentences_length):
        embedded_sentences = self.embedding_layer(sentences)
        packed_sentences = nn.utils.rnn.pack_padded_sequence(embedded_sentences, sentences_length, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.bidirectional_layer_1(packed_sentences)
        packed_output, _ = self.bidirectional_layer_2(packed_output)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        output = self.dense_layer(output)
        output = F.log_softmax(output, dim=2)
        return output

### Model 2

In [12]:
class Model2(nn.Module):
    def __init__(self, lstm_dimension, dense_dimension):
        super().__init__()
        self.bidirectional_layer = nn.LSTM(bidirectional=True, input_size=embedding_dimension, hidden_size=lstm_dimension, batch_first=True)
        self.dense_layer_1 = nn.Linear(in_features=lstm_dimension*2, out_features=dense_dimension)
        self.dense_layer_2 = nn.Linear(in_features=dense_dimension, out_features=dense_dimension)
        self.embedding_layer, self.embedding_dim = create_emb_layer(embedding_matrix)

    def forward(self, sentences, sentences_length):
        embedded_sentences = self.embedding_layer(sentences)
        packed_sentences = nn.utils.rnn.pack_padded_sequence(embedded_sentences, sentences_length, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.bidirectional_layer(packed_sentences)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        output = self.dense_layer_1(output)
        output = self.dense_layer_2(output)
        output = F.log_softmax(output, dim=2)
        return output

# [Task 4 - 1.0 points] Metrics

Before training the models, you are tasked to define the evaluation metrics for comparison.

### Instructions

* Evaluate your models using macro F1-score, compute over **all** tokens.
* **Concatenate** all tokens in a data split to compute the F1-score. (**Hint**: accumulate FP, TP, FN, TN iteratively)
* **Do not consider punctuation and symbol classes** $\rightarrow$ [What is punctuation?](https://en.wikipedia.org/wiki/English_punctuation)

**Note**: What about OOV tokens?
   * All the tokens in the **training** set that are not in GloVe are **not** considered as OOV
   * For the remaining tokens (i.e., OOV in the validation and test sets), you have to assign them a **static** embedding.
   * You are **free** to define the static embedding using any strategy (e.g., random, neighbourhood, etc...)

In [13]:
from sklearn.metrics import f1_score

def accuracy_and_f1(y_pred, y_true):
    correct = y_pred.eq(y_true)          
    acc = correct.sum()/y_true.shape[0] 
    f1 = f1_score(y_true,y_pred,average='macro')
    return acc,f1

### Let's check our OOV tokens

In [14]:
def check_OOV_terms(embedding_model, unique_words, lower):
    oov_words = []
    int_oov_words = []

    if lower:
        words = set([x.lower() for x in unique_words])
    else: 
        words = unique_words

    for word in words:
        if word not in embedding_model.itos:
           oov_words.append(word) 
           int_oov_words.append(word2int[word]) 
    
    print("Total number of unique words in dataset:",len(words))
    print(oov_words)
    print("Total OOV terms: {0} ({1:.2f}%)".format(len(oov_words), (float(len(oov_words)) / len(words))*100))
    print("Some OOV terms:",random.sample(oov_words,15))
    
    return oov_words, int_oov_words

oov_words, int_oov_words = check_OOV_terms(glove_embeddings, unique_words,True)

Total number of unique words in dataset: 10947
['2691.19', 'language-housekeeper', '372.9', '95.09', '13\\/16', 'wheel-loader', 'news-american', '1206.26', 'chilver', 'pension-fund', 'ingersoll-rand', '-lcb-', 'hart-scott-rodino', 'crane-safety', 'intellectual-property', 'boogieman', 'life-of-contract', 'pro-forma', 'chemplus', '7\\/16', 'futures-related', 'marketing-communications', '237-seat', 'car-development', 'chong-sik', 'non-biodegradable', 'recession-inspired', 'phacoflex', 'tete-a-tete', 'propagandizes', 'stock-index', 'bell-ringer', 'shokubai', 'collective-bargaining', 'prudential-bache', '35564.43', 'profit-taking', 'quantitive', 'asset-valuation', '188.84', 'several-year', 'nylev', '35500.64', 'stock-selection', '100,980', '11-month-old', '34.625', 'trettien', 'pro-iranian', 'heebie-jeebies', 'ghkm', '29year', 'million-a-year', '5.435', 'yen-support', 'anti-programmers', 'arbitrage-related', 'uzi-model', 'headcount-control', '82,389', 'red-flag', '1.637', 'enzor', 'savings-

As we can see the number of OOV words is very high (31.29% of the total words in the corpus). So, we have to find a way to deal with them.<br>
A good idea could be to use lowercased words, in fact in this way the number of OOV decreases to 6.18%, but we have to be careful because we could lose some information. <br>
For example, the word "Pierre" is a name, but "pierre" is a noun. So, we have to find a way to deal with this problem.<br>
We decided to test the model with the two different approaches and see which one is better.

# [Task 5 - 1.0 points] Training and Evaluation

You are now tasked to train and evaluate the Baseline, Model 1, and Model 2.

### Instructions

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Compute metrics on the validation set.
* Pick **at least** three seeds for robust estimation.
* Pick the **best** performing model according to the observed validation set performance.

In [15]:
def initialize_weights(model):
    for _, param in model.named_parameters():
        if isinstance(model, nn.LSTM) or isinstance(model, nn.Linear):
            nn.init.normal_(param.data, mean = 0, std = 0.1)

In [16]:
def number_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [17]:
def get_to_be_masked_tags():
    punctuation_tags = ['$', '``', '.', ',', '#', 'SYM', ':', "''",'-RRB-','-LRB-']   #tags to be masked 
    token_punctuation = [tag2int[tag] for tag in punctuation_tags]
    return torch.LongTensor(token_punctuation+[0])

to_mask = get_to_be_masked_tags()

def reshape_and_mask(predictions,targets): 
    non_masked_elements = torch.isin(targets, to_mask, invert=True)
    
    return predictions[non_masked_elements],targets[non_masked_elements]


In [18]:
from torch.utils.data import Dataset, DataLoader

class PosDataset(Dataset):
    def __init__(self, text, labels):
        self.labels = labels
        self.text = text
        self.sentence_lengths = [len(sentence) for sentence in self.text]
    def __len__(self):
            return len(self.labels)
    def __getitem__(self, idx):
            label = self.labels[idx]
            text = self.text[idx]
            sample = (text, label, self.sentence_lengths[idx])
            return sample


def collate_fn(data):
    return ([x[0] for x in data], [x[1] for x in data], [x[2] for x in data])


def create_dataloaders(b_s : int):
    train_df = indexed_dataset[indexed_dataset['split'] == 'train'].reset_index(drop=True)      
    val_df = indexed_dataset[indexed_dataset['split'] == 'val'].reset_index(drop=True)
    test_df = indexed_dataset[indexed_dataset['split'] == 'test'].reset_index(drop=True)

    #create DataframeDataset objects for each split 
    train_dataset = PosDataset(train_df.iloc[:,2],train_df.iloc[:,3])
    val_dataset = PosDataset(val_df.iloc[:,2],val_df.iloc[:,3])
    test_dataset = PosDataset(test_df.iloc[:,2],test_df.iloc[:,3])

    train_dataloader = DataLoader(train_dataset, batch_size=b_s, shuffle=True, collate_fn= collate_fn)
    val_dataloader = DataLoader(val_dataset, batch_size=b_s, shuffle=True, collate_fn= collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=b_s, shuffle=True, collate_fn= collate_fn)

    return train_dataloader,val_dataloader,test_dataloader 

In [19]:
batch_size = 32

tr_dl, val_dl, test_dl = create_dataloaders(batch_size)

In [29]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import torch.nn.utils.rnn as rnn
import time

def train(model, epochs, loss_function, dataloader, optimizer, padding_value = 0, seed):
    model.train()
    for epoch in range(epochs):
        start_time = time.time()
        for sentences, pos, s_len in dataloader:
            optimizer.zero_grad()
            
            tensor_sentences = [torch.LongTensor(s) for s in sentences]
            tensor_pos = [torch.LongTensor(p) for p in pos]

            padded_sentences = rnn.pad_sequence(tensor_sentences, batch_first = True, padding_value = 0)
            padded_pos = rnn.pad_sequence(tensor_pos, batch_first = True, padding_value=padding_value)

            predicted = model(padded_sentences, s_len)

            predicted = predicted.view(-1,predicted.shape[-1])    
            targets = padded_pos.view(-1)

            predicted, targets = reshape_and_mask(predicted, targets)

            loss = loss_function(predicted, targets)
            loss.backward()
            optimizer.step()
        elapsed = time.time() - start_time 
        print(f'Train epoch [{epoch+1}/{epochs}] loss: {loss.item()} time: {elapsed:.2f}s')

def evaluate(model, loss_function, dataloader, padding_value=0):
    model.eval()
    tot_pred , tot_targ = torch.LongTensor(), torch.LongTensor()
    epoch_loss = 0
    for sentences, pos, s_len in dataloader:
        tensor_sentences = [torch.LongTensor(s) for s in sentences]
        tensor_pos = [torch.LongTensor(p) for p in pos]

        padded_sentences = rnn.pad_sequence(tensor_sentences, batch_first = True, padding_value = 0)
        padded_pos = rnn.pad_sequence(tensor_pos, batch_first = True, padding_value=padding_value)

        predicted = model(padded_sentences, s_len)
        predicted = predicted.view(-1,predicted.shape[-1])    
        targets = padded_pos.view(-1)

        predicted, targets = reshape_and_mask(predicted, targets)

        loss = loss_function(predicted, targets)

        predicted = predicted.argmax(dim=1)

        tot_pred = torch.cat((tot_pred,predicted))
        tot_targ = torch.cat((tot_targ,targets))

        epoch_loss += loss.item()
    full_accuracy, full_f1 = accuracy_and_f1(tot_pred,tot_targ)
    print(f'Eval: loss: {epoch_loss:.2f} accuracy: {full_accuracy:.2f} f1: {full_f1:.2f}')
    return full_accuracy,full_f1,tot_pred,tot_targ


In [21]:
loss_function = CrossEntropyLoss()


lstm_dimension = 16
dense_dimension = len(unique_tags)+1

baseline_model = Baseline(lstm_dimension, dense_dimension)
baseline_model.apply(initialize_weights)

double_lstm_model = Model1(lstm_dimension, dense_dimension)
double_lstm_model.apply(initialize_weights)

double_dense_model = Model2(lstm_dimension, dense_dimension)
double_dense_model.apply(initialize_weights)

Model2(
  (bidirectional_layer): LSTM(300, 16, batch_first=True, bidirectional=True)
  (dense_layer_1): Linear(in_features=32, out_features=46, bias=True)
  (dense_layer_2): Linear(in_features=46, out_features=46, bias=True)
  (embedding_layer): Embedding(10948, 300, padding_idx=0)
)

In [22]:
print(f'Number of parameters in baseline model: {number_parameters(baseline_model)}')
print(f'Number of parameters in double lstm model: {number_parameters(double_lstm_model)}')
print(f'Number of parameters in double dense model: {number_parameters(double_dense_model)}')

Number of parameters in baseline model: 42222
Number of parameters in double lstm model: 48622
Number of parameters in double dense model: 44384


In [23]:
weight_folder = "./weights"
def save_weights(model, model_name):
    """
        Saves the model weights in the folder "weights"
    """
    path = os.path.join(weight_folder, model_name)
    torch.save(model.state_dict(), path)

def load_weights(model, model_name):
    """
        Loads the model weights from the folder "weights"
    """
    path = os.path.join(weight_folder, model_name)
    model.load_state_dict(torch.load(path))

In [24]:
epochs = 50
optimizer = Adam(baseline_model.parameters(), lr=5e-4)

train(baseline_model, epochs, loss_function, tr_dl, optimizer)

Train epoch [1/50] loss: 3.017608404159546 time: 3.67s
Train epoch [2/50] loss: 2.545351266860962 time: 3.01s
Train epoch [3/50] loss: 2.2228682041168213 time: 2.97s
Train epoch [4/50] loss: 2.0715620517730713 time: 3.02s
Train epoch [5/50] loss: 1.9211663007736206 time: 3.08s
Train epoch [6/50] loss: 1.6415138244628906 time: 3.01s
Train epoch [7/50] loss: 1.524673342704773 time: 2.68s
Train epoch [8/50] loss: 1.443713665008545 time: 2.89s
Train epoch [9/50] loss: 1.3459240198135376 time: 2.88s
Train epoch [10/50] loss: 1.067432165145874 time: 3.21s
Train epoch [11/50] loss: 1.2001962661743164 time: 3.16s
Train epoch [12/50] loss: 0.9225680232048035 time: 3.08s
Train epoch [13/50] loss: 0.9394091367721558 time: 3.33s
Train epoch [14/50] loss: 0.8425342440605164 time: 2.94s
Train epoch [15/50] loss: 0.8610190749168396 time: 3.02s
Train epoch [16/50] loss: 0.9595901966094971 time: 3.05s
Train epoch [17/50] loss: 0.6177699565887451 time: 2.71s
Train epoch [18/50] loss: 0.9604779481887817 

In [25]:
epochs = 50
optimizer = Adam(double_lstm_model.parameters(), lr=5e-4)

train(double_lstm_model, epochs, loss_function, tr_dl, optimizer)

Train epoch [1/50] loss: 3.01688289642334 time: 5.85s
Train epoch [2/50] loss: 2.6593449115753174 time: 5.29s
Train epoch [3/50] loss: 2.752267837524414 time: 5.81s
Train epoch [4/50] loss: 2.7615883350372314 time: 5.22s
Train epoch [5/50] loss: 2.5704667568206787 time: 5.13s
Train epoch [6/50] loss: 2.2798640727996826 time: 5.96s
Train epoch [7/50] loss: 1.9929006099700928 time: 5.38s
Train epoch [8/50] loss: 1.7936513423919678 time: 5.33s
Train epoch [9/50] loss: 1.795961856842041 time: 5.14s
Train epoch [10/50] loss: 1.4651871919631958 time: 5.21s
Train epoch [11/50] loss: 1.4048948287963867 time: 5.26s
Train epoch [12/50] loss: 1.2603811025619507 time: 6.00s
Train epoch [13/50] loss: 1.2478976249694824 time: 5.45s
Train epoch [14/50] loss: 1.163589596748352 time: 5.30s
Train epoch [15/50] loss: 0.9240286946296692 time: 5.44s
Train epoch [16/50] loss: 0.9154030084609985 time: 5.25s
Train epoch [17/50] loss: 1.0240864753723145 time: 5.56s
Train epoch [18/50] loss: 0.6122284531593323 

In [26]:
epochs = 50
optimizer = Adam(double_dense_model.parameters(), lr=5e-4)

train(double_dense_model, epochs, loss_function, tr_dl, optimizer)

Train epoch [1/50] loss: 2.7972629070281982 time: 3.19s
Train epoch [2/50] loss: 2.3723249435424805 time: 3.19s
Train epoch [3/50] loss: 1.8511394262313843 time: 3.08s
Train epoch [4/50] loss: 1.5592678785324097 time: 3.26s
Train epoch [5/50] loss: 1.351058840751648 time: 3.08s
Train epoch [6/50] loss: 1.2130072116851807 time: 2.81s
Train epoch [7/50] loss: 1.0376696586608887 time: 3.13s
Train epoch [8/50] loss: 0.835885226726532 time: 2.98s
Train epoch [9/50] loss: 0.7687786221504211 time: 3.44s
Train epoch [10/50] loss: 0.8114917874336243 time: 3.12s
Train epoch [11/50] loss: 0.6615330576896667 time: 3.32s
Train epoch [12/50] loss: 0.7295700907707214 time: 3.26s
Train epoch [13/50] loss: 0.3838130533695221 time: 3.35s
Train epoch [14/50] loss: 0.6530866622924805 time: 3.06s
Train epoch [15/50] loss: 0.43102726340293884 time: 3.08s
Train epoch [16/50] loss: 0.7163329124450684 time: 3.16s
Train epoch [17/50] loss: 0.5856622457504272 time: 2.98s
Train epoch [18/50] loss: 0.4014804661273

In [30]:
baseline_accuracy, baseline_f1, baseline_pred, baseline_targ = evaluate(baseline_model, loss_function, val_dl)

Eval: loss: 18.15 accuracy: 0.88 f1: 0.67


In [31]:
double_lstm_accuracy, double_lstm_f1, double_lstm_pred, double_lstm_targ = evaluate(double_lstm_model, loss_function, val_dl)

Eval: loss: 16.87 accuracy: 0.89 f1: 0.65


In [32]:
double_dense_accuracy, double_dense_f1, double_dense_pred, double_dense_targ = evaluate(double_dense_model, loss_function, val_dl)

Eval: loss: 16.50 accuracy: 0.89 f1: 0.74


# [Task 6 - 1.0 points] Error Analysis

You are tasked to evaluate your best performing model.

### Instructions

* Compare the errors made on the validation and test sets.
* Aggregate model errors into categories (if possible)
* Comment the about errors and propose possible solutions on how to address them.

# [Task 7 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...
* You can upload **model weights** in a cloud repository and report the link in the report.

# FAQ

Please check this frequently asked questions before contacting us

### Trainable Embeddings

You are **free** to define a trainable or non-trainable Embedding layer to load the GloVe embeddings.

### Model architecture

You **should not** change the architecture of a model (i.e., its layers).

However, you are **free** to play with their hyper-parameters.

### Neural Libraries

You are **free** to use any library of your choice to implement the networks (e.g., Keras, Tensorflow, PyTorch, JAX, etc...)

### Keras TimeDistributed Dense layer

If you are using Keras, we recommend wrapping the final Dense layer with `TimeDistributed`.

### Error Analysis

Some topics for discussion include:
   * Model performance on most/less frequent classes.
   * Precision/Recall curves.
   * Confusion matrices.
   * Specific misclassified samples.

### Punctuation

**Do not** remove punctuation from documents since it may be helpful to the model.

You should **ignore** it during metrics computation.

If you are curious, you can run additional experiments to verify the impact of removing punctuation.

# The End